# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Импортировали pandas, открыли датасет, сохранили в переменную df, напечатали верхние пять строк таблицы. Названия столбцов нормальные.

Очевидные проблемы:
* нет колонки с id клиента;
* отрицательные дни трудоустройства, еще и дробные; значения тоже непонятные;
* по-разному обозначено образование (проблемы с регистром, минимум).

In [2]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Вывели сводную информацию о датасете. Непонятно, зачем дни трудоустройства в формате float. Total_income c точностью до милионных долей рубля - тоже плохая идея. Нужно будет заменить эти типы на integer. Очевидно, есть пропуски в днях трудоустройства и общем доходе.

Проверяем наличие явных дубликатов методом duplicated():

In [3]:
df.duplicated().sum()

54

Обнаружено 54 явных дубликата, убираем их и проверяем, что дубликатов не осталось:

In [4]:
df = df.drop_duplicates()
df.duplicated().sum()

0

Добавим id клиента.

In [5]:
df.insert(0, 'id', list(range(len(df))))
display(df.head())

,id,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 13 columns):
id                  21471 non-null int64
children            21471 non-null int64
days_employed       19351 non-null float64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        19351 non-null float64
purpose             21471 non-null object
dtypes: float64(2), int64(6), object(5)
memory usage: 2.3+ MB


None

**Вывод**

Датафрейм импортировался успешно, но данные нуждаются в предобработке: заполнении пропусков, удалениинеявных дубликатов, унификации значений, замене типа данных для `days_employed`.
    

## Шаг 2. Предобработка данных

### Обработка пропусков

Выведем названия столбцов.

In [7]:
df.columns

Index(['id', 'children', 'days_employed', 'dob_years', 'education',
       'education_id', 'family_status', 'family_status_id', 'gender',
       'income_type', 'debt', 'total_income', 'purpose'],
      dtype='object')

Проверим все столбцы на наличие пропусков:

In [8]:
df.isna().sum()

id                     0
children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` не хватает 2120 записи в каждом, это примерно 9,9% выборки. Наверное, данных нет в одних и тех же рядах. Проверка ниже подтверждает догадку: `id` клиентов, у которых нет информации по `days_employed`, совпадают с `id` клиентов, у которых нет информации по `total_income`. 

In [9]:
(df[df['days_employed'].isna() == True]['id'] == df[df['total_income'].isna() == True]['id']).unique()

array([ True])

Причины пропуска данных - вероятно, этих данных просто не было по этим клиентам.

Избавимся от пропусков.

Данные по дням трудоустройства вообще неправдоподобные, пенсионер `df.loc[4]` проработал 932 года, если это дни. Там есть пропуски, отрицательные значения, дробные значения. В задании эта колонка не нужна, не вижу смысла приводить ее в порядок, удаляю вместе с пропусками:

In [10]:
df = df.drop('days_employed', axis=1)
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Избавимся от пропусков в колонке `total_income`. Заполним их медианным значением по столбцу. Для женщин заполним женской медианой, для мужчин - мужской. 

Сперва проверим, сколько уникальных гендеров в датасете, а также распределение по полам:

In [11]:
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Все гендеры указаны в верхнем регистре, приводить к единому не надо. Зато есть один неопределившийся. 

Проверим неопределившегося: все данные по нему есть, пол нам в исследовании не важен. Не удаляю.

In [12]:
df[df['gender'] == 'XNA']

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,10690,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Посчитаем медианные значения дохода:

In [13]:
female_median_income = df[df['gender'] == 'F']['total_income'].median()
male_median_income = df[df['gender'] == 'M']['total_income'].median()
median_income = df['total_income'].median()
print(female_median_income)
print(male_median_income)
print(median_income)


134155.2834794494
167714.34371587454
145017.93753253992


Заполним пропуски в `df['total_income']` медианными значениями:

In [14]:
import numpy as np
df.loc[(df['gender'] == 'F') & (np.isnan(df['total_income'])), 'total_income'] = female_median_income
df.loc[(df['gender'] == 'M') & (np.isnan(df['total_income'])), 'total_income'] = male_median_income

In [15]:
def custom_median_income(person):
    gender = person['gender']
    education_id = person['education_id']
    income_type = person['income_type']
    custom_median_income = df[(df['gender'] == gender) & 
                              (df['income_type'] == income_type) & 
                              (df['education_id'] == education_id)
                             ]['total_income'].median()
    return custom_median_income
# Что-нибудь такое, но это набросок.

In [17]:
df1 = df.copy()
df1.education = df1.education.str.lower()

df1.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')

education                      высшее      начальное  неоконченное высшее  \
income_type     gender                                                      
безработный     F       202722.511368            NaN                  NaN   
                M                 NaN            NaN                  NaN   
в декрете       F                 NaN            NaN                  NaN   
госслужащий     F       145017.937533  119960.843390        134155.283479   
                M       204717.884325  190966.659534        167714.343716   
компаньон       F       174396.822139  134083.643523        164996.813904   
                M       218263.636723  150100.960964        179370.487646   
                XNA               NaN            NaN        203905.157261   
пенсионер       F       134155.283479  102598.653164        130837.388717   
                M       166620.123422  114068.787524        124667.471301   
предприниматель F       499163.144947            NaN                  NaN   
                M       167714.343716            NaN                  NaN   
сотрудник       F       144239.086207  124397.467932        134155.283479   
                M       179652.712804  147989.228695        175068.921286   
студент         M        98201.625314            NaN                  NaN   

education                     среднее  ученая степень  
income_type     gender                                 
безработный     F                 NaN             NaN  
                M        59956.991984             NaN  
в декрете       F        53829.130729             NaN  
госслужащий     F       134155.283479   111392.231107  
                M       167714.343716             NaN  
компаньон       F       136301.401814             NaN  
                M       170206.988773             NaN  
                XNA               NaN             NaN  
пенсионер       F       122776.401763   255425.196556  
                M       134353.774426    98752.495442  
предприниматель F                 NaN             NaN  
                M                 NaN             NaN  
сотрудник       F       132162.260689             NaN  
                M       163894.312122   198570.757322  
студент         M                 NaN             NaN

In [18]:
qq = df1.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')
qq.loc[('студент', 'M')]['высшее']

98201.62531401133

In [19]:
def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


In [20]:
print(super_fillna_func('ревьюер', 'F','высшее') )

0


In [21]:
# Так сработает apply.
df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

0        144239.086207
1        132162.260689
2        163894.312122
3        163894.312122
4        122776.401763
             ...      
21520    136301.401814
21521    122776.401763
21522    163894.312122
21523    163894.312122
21524    132162.260689
Length: 21471, dtype: float64

In [ ]:
# Вот так применяем к таблице.

# Запишем в новый столбец.
df1['new_income'] = df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

# Пандас сам заменит пропуски значениями из нового столбца в той же строке.
df1['total_income'] = df1['total_income'].fillna(df1['new_income'])

In [ ]:
# А вот так мы столбец создадим, но сохранять не будем.
df1['total_income'] = df1['total_income'].fillna(df1.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1))

Проверим, что в таблице отсутствуют пропуски:

In [300]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
id                  21471 non-null int64
children            21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float64
purpose             21471 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


None

**Вывод**

В таблице больше нет пропусков данных: колонку `days_employed` убрали вместе с пропусками, a в колонке `total_income` пропуски заполнили медианными значениями.

### Замена типа данных

Заменим тип колонки `total_income`  на целочисленный, т.к. тип `float` предполагает избыточную точность (которая еще и не соответствует реальности - сейчас это милионные доли рубля). Наличие долга - по идее булева переменная, но представлена типом `integer` - тоже заменю. Делаю замену:

In [301]:
df['total_income'] = df['total_income'].astype('int')
df['debt'] = df['debt'].astype('bool')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 12 columns):
id                  21471 non-null int64
children            21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null bool
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: bool(1), int64(6), object(5)
memory usage: 2.0+ MB


Замена проведена методом `.astype()`, потому что он здесь подходит - позволяет задать тип, в который нужно конвертировать. Метод `.to_numeric()` конвертирует из строки в `float`, поэтому здесь не подходит.

**Вывод**

Заменили тип данных колонки `total_income` на `integer`, а колонки `debt` - на `boolean`.

### Обработка дубликатов

Проверим данные на наличие дубликатов. До этого проверим уникальные значения в столбцах `education`, `family_status`, `income_type`, `purpose` на наличие неявных дубликатов.

In [302]:
print(df['education'].sort_values().unique())
print(df['family_status'].sort_values().unique())
print(df['income_type'].sort_values().unique())
print(df['purpose'].sort_values().unique())

['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']
['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']
['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']
['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своег

В столбце `education` есть данные, которые отличаются только регистром, конвертирую все в нижний регистр:

In [303]:
df['education'] = df['education'].str.lower()
print(df['education'].sort_values().unique())

['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']


В столбце `purpose` много целей, которые по-разному сформулированы, но по сути означают одно. Их будем выявлять через лемматизацию. В столбцах `family_status` и `income_type` неявных дубликатов нет.

**Вывод**

Дубликатов больше нет.

### Лемматизация

Лемматизируем данные в столбце `purpose`. Для этого импортируем Mystem из яндексовской библиотеки pymystem3.

In [304]:
from pymystem3 import Mystem
m = Mystem()

lemmas_list = []

for purpose in df['purpose']:
    lemmas = m.lemmatize(purpose)
    lemmas_list.append(lemmas)

print(lemmas_list)

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилье', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['недвижимость', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'подержа

Добавили колонку `purpose_lemmas` в датасет:

In [305]:
df['purpose_lemmas'] = lemmas_list
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


**Вывод**

Лемматизация проведена. Результатом воспользуемся для категоризации данных.

### Категоризация данных

Посмотрим самые частотные слова в `purpose`, чтоб на их основании сформировать категории. Для этого воспользуемся словарем Counter. Импортируем его и передадим ему леммы, приведенные в вид одномерного массива.

In [306]:
from collections import Counter
flat_lemmas_list = [x for sublist in lemmas_list for x in sublist]
Counter(flat_lemmas_list).most_common()


[(' ', 33596),
 ('\n', 21471),
 ('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2335),
 ('свой', 2231),
 ('на', 2228),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315),
 ('коммерческий', 1312),
 ('для', 1290),
 ('жилой', 1231),
 ('сделка', 941),
 ('дополнительный', 907),
 ('заниматься', 904),
 ('проведение', 773),
 ('сыграть', 769),
 ('сдача', 652),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Выглядит так, будто основные цели кредитования - недвижимость (жилье), покупка автомобиля, получение образования, свадьба. Так как наиболее часто встречающиеся в леммах значимые слова - "недвижимость", "жилье", "автомобиль", "образование", "свадьба". 

Однако неясно, возможно, наличие слов "ремонт", "операция" в сочетании с "автомобилем", "недвижимостью", "жильем" изменит значение цели. 

Пишу функцию для проверки. Функция принимает на вход слово, а возвращает все леммы, где есть это слово.

In [307]:
def checker(word):
    checklist = []
    for lemma in lemmas_list:
        for el in lemma:
            if el == word:
                checklist.append(lemma)
    return(checklist)
    

Проверка лемм со словом "ремонт" показывает: все кредиты, взятые на ремонт, взяты на ремонт жилья. Выделим его в отдельную категорию.

In [308]:
checker('ремонт')

[['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт', ' ', 'жилье', '\n'],
 ['ремонт'

Проверка слова "операция" показывает, что все операции - это операции с недвижимостью. Т.е. слово "операция" ничего не добавляет к знанию о цели сделки. Слово можно игнорировать.

In [309]:
checker('операция')

[['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],


К счастью, выборка данных маленькая, результаты работы функции `checker(word)` можно просмотреть глазом. Если бы выборка была больше и разнообразней, потребовалась бы более сложная проверка. Но для этой выборки простой проверки достаточно.

Категоризируем цели получения кредита. Для этого напишем функцию `purpose_group`, которая принимает на вход лемму цели, а возвращает категорию цели. 

In [310]:
def purpose_group(purpose_lemma):
    """
    Функция возвращает результаты:
    "ремонт жилья" - если в лемме есть слово "ремонт";
    "недвижимость" - если в лемме нет слова "ремонт", но есть слова "жилье", "недвижимость";
    "получение образования" - если в лемме есть слово "образование";
    "свадьба" - если в лемме есть слово свадьба
    "автомобиль" - если в лемме есть слово "автомобиль"
    """
    
    if "ремонт" in purpose_lemma:
        return "ремонт жилья"
    elif "жилье" in purpose_lemma or "недвижимость" in purpose_lemma:
        return "недвижимость"
    elif "образование" in purpose_lemma:
        return "получение образования"
    elif "свадьба" in purpose_lemma:
        return "свадьба"
    elif "автомобиль" in purpose_lemma:
        return "автомобиль"
    else:
        return "неизвестно"
    

Добавим в датасет колонку `purpose_group`, в которой сохраним категорию цели кредита.

In [311]:
df['purpose_group'] = df['purpose_lemmas'].apply(purpose_group)
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]",получение образования
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


**Вывод**

На основе лемматизации данных мы выделили основные цели кредитов: "недвижимость", "ремонт жилья", "получение образования", "автомобиль", "свадьба". Выделенные категории записали в новый столбец датасета `purpose_group`.

Создадим справочную таблицу.

In [312]:
purpose_group_dict = pd.DataFrame(
    {'name': ["ремонт жилья", 
              "недвижимость", 
              "получение образования",  
              "свадьба", 
              "автомобиль", 
              "неизвестно"]}
)
purpose_group_dict.insert(0, 'id', list(range(len(purpose_group_dict))))
display(purpose_group_dict)


,id,name
0,0,ремонт жилья
1,1,недвижимость
2,2,получение образования
3,3,свадьба
4,4,автомобиль
5,5,неизвестно


Пишем функцию, чтоб она возвращала id вместо имени группы:

In [313]:
def purpose_group_id(purpose_lemma):
    """
    Функция возвращает результаты:
    id для "ремонт жилья" - если в лемме есть слово "ремонт";
    id для "недвижимость" - если в лемме нет слова "ремонт", но есть слова "жилье", "недвижимость";
    id для "получение образования" - если в лемме есть слово "образование";
    id для "свадьба" - если в лемме есть слово свадьба
    id для "автомобиль" - если в лемме есть слово "автомобиль"
    id для "неизвестно" - если ни одна из категорий не подходит
    """
    
    if "ремонт" in purpose_lemma:
        return int(purpose_group_dict[purpose_group_dict['name'] == "ремонт жилья"]['id'])        
    elif "жилье" in purpose_lemma or "недвижимость" in purpose_lemma:
        return int(purpose_group_dict[purpose_group_dict['name'] == "недвижимость"]['id'])
    elif "образование" in purpose_lemma:
        return int(purpose_group_dict[purpose_group_dict['name'] == "получение образования"]['id'])
    elif "свадьба" in purpose_lemma:
        return int(purpose_group_dict[purpose_group_dict['name'] == "свадьба"]['id'])
    elif "автомобиль" in purpose_lemma:
        return int(purpose_group_dict[purpose_group_dict['name'] == "автомобиль"]['id'])
    else:
        return int(purpose_group_dict[purpose_group_dict['name'] == "неизвестно"]['id'])
    
# Я завернула в `int` это: int(purpose_group_dict[purpose_group_dict['name'] == "неизвестно"]['id']),
# поскольку: 
# type(purpose_group_dict[purpose_group_dict['name'] == "неизвестно"]['id']) возвращает pandas.core.series.Series
# соответственно, возникает ошибка `ValueError: Wrong number of items passed 5, placement implies` 
# при вызове функции в ячейке 102: попытка записать Series в элемент pandas df.


Добавляем в `df` столбец `purpose_id`, и в этом случае не надо было бы добавлять столбец `purpose_group`, 
потому что название группы хранится в справочной таблице `purpose_group_dict`:

In [314]:
df['purpose_id'] = df['purpose_lemmas'].apply(purpose_group_id)
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group,purpose_id
0,0,1,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1
1,1,1,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,4
2,2,0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1
3,3,3,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]",получение образования,2
4,4,0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,3


## Шаг 3. Ответьте на вопросы

### Посчитаем средний % должников по всей выборке:

In [315]:
avg_debt_percent = round(df['debt'].sum() / df['debt'].count() * 100, 2)
print(avg_debt_percent)

8.11


### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем имеющихся заемщиков по наличию детей, чтоб в дальнейшем разделить их на группы по этому признаку.

In [316]:
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицательные дети - явная ошибка в данных. Избавимся от нее. Мы не знаем, что за этим стоит: ноль детей, один ребенок, еще что-то. Поэтому заменяем на тип `NaN` и исключаем эти строки из рассмотрения. (А можно было просто удалить их: строк с ошибочными данными всего 0.22%).

In [317]:
import numpy as np
df['children'] = df['children'].replace(-1, np.nan)
print(df['children'].unique())
df['children'].value_counts()

[ 1.  0.  3.  2. nan  4. 20.  5.]


0.0     14107
1.0      4809
2.0      2052
3.0       330
20.0       76
4.0        41
5.0         9
Name: children, dtype: int64

20 детей тоже может быть ошибкой в данных, учитывая, что в выборке нет семей с детьми в количестве от 6 до 19, а с 20 детьми - целых 76. Привожу решение, где я верю в то, что данные верные, хотя можно, наверное, исключить эти строки из рассмотрения (более надежный вариант) или предположить, что в этих семьях на самом деле 2 ребенка, а не 20. 

Разделим семьи на группы по количеству детей. Для этого напишем функцию `children_group`, которая принимает на вход количество детей и возвращает группу.

In [318]:
def children_group(number):
    if number == 0:
        return 'нет детей'
    elif number == 1:
        return 'один ребенок'
    elif number == 2:
        return 'два ребенка'
    elif number == 3:
        return 'три ребенка'
    elif number > 3 and number <= 10:
        return 'от четырех до десяти детей'
    elif number > 10:
        return 'свыше десяти детей'
    else:
        return 'неизвестно'

Создадим новую колонку в датасете, в которую внесем данные о группе по количеству детей.

In [319]:
df['children_group'] = df['children'].apply(children_group)
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group,purpose_id,children_group
0,0,1.0,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,один ребенок
1,1,1.0,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,4,один ребенок
2,2,0.0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,нет детей
3,3,3.0,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]",получение образования,2,три ребенка
4,4,0.0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,3,нет детей


Сгруппируем заемщиков по количеству детей и вычислим процент должников в каждой группе. Должников с неизвестным количеством детей проигнорируем.

In [320]:
no_children_debt_percent = round(df[df['children_group'] == 'нет детей']['debt'].sum() / df[df['children_group'] == 'нет детей']['debt'].count() * 100, 2)
print(str(no_children_debt_percent) + ' - % должников без детей')

one_child_debt_percent = round(df[df['children_group'] == 'один ребенок']['debt'].sum() / df[df['children_group'] == 'один ребенок']['debt'].count() * 100, 2)
print(str(one_child_debt_percent) + ' - % должников с одним ребенком')

two_children_debt_percent = round(df[df['children_group'] == 'два ребенка']['debt'].sum() / df[df['children_group'] == 'два ребенка']['debt'].count() * 100, 2)
print(str(two_children_debt_percent) + ' - % должников с двумя детьми')

three_children_debt_percent = round(df[df['children_group'] == 'три ребенка']['debt'].sum() / df[df['children_group'] == 'три ребенка']['debt'].count() * 100, 2)
print(str(three_children_debt_percent) + ' - % должников с тремя детьми')

more_than_four_children_debt_percent = round(df[df['children_group'] == 'от четырех до десяти детей']['debt'].sum() / df[df['children_group'] == 'от четырех до десяти детей']['debt'].count() * 100, 2)
print(str(more_than_four_children_debt_percent) + ' - % должников с четырьмя и более детьми')

more_than_ten_children_debt_percent = round(df[df['children_group'] == 'свыше десяти детей']['debt'].sum() / df[df['children_group'] == 'свыше десяти детей']['debt'].count() * 100, 2)
print(str(more_than_ten_children_debt_percent) + ' - % должников, имеющих свыше десяти детей')

7.54 - % должников без детей
9.23 - % должников с одним ребенком
9.45 - % должников с двумя детьми
8.18 - % должников с тремя детьми
8.0 - % должников с четырьмя и более детьми
10.53 - % должников, имеющих свыше десяти детей


In [321]:
df.groupby('children_group')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children_group,,,
два ребенка,2052,194.0,9.45%
неизвестно,47,1.0,2.13%
нет детей,14107,1063.0,7.54%
один ребенок,4809,444.0,9.23%
от четырех до десяти детей,50,4.0,8.00%
свыше десяти детей,76,8.0,10.53%
три ребенка,330,27.0,8.18%


**Вывод**

Помня, что средний процент должников по выборке 8.11 %, можем сделать вывод, что количество детей влияет на возврат кредита в срок. 

Выгоднее всего давать кредит заемщикам без детей (7.54 % должников - это меньше, чем в среднем). 

Риск невозврата кредита в срок повышается для заемщиков с одним и заемщиков двумя детьми и составляет 9.23 % и 9.45 % соответственно. 

Для заемщиков с тремя - пятью детьми риск задолженности снижается и приближается  к среднему по всей выборке (8.18% для семей с тремя детьми, 8.0% для семей с четырьмя-пятью детьми - мы знаем, что в группу "от четырех до десяти детей" попали именно они).

Для заемщиков, имеющих свыше десяти детей (а именно, 20 детей применительно к заемщикам из выборки) процент должников составляет 10.53 %, то есть это наиболее рискованная группа заемщиков. Возможно, эта группа отличается не только количеством детей, но и по социальному или национальному признаку. А возможно, в этих семьях на самом деле 2 ребенка, и тогда ухудшается статистика для заемщиков с двумя детьми.


### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выведем возможные варианты семейного положения:

In [322]:
print(df['family_status'].sort_values().unique())

['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']


Сгруппируем по этим вариантам выборку и вычислим % должников. 

In [323]:
df.groupby('family_status')['debt'].agg([lambda x: (str(round(x.mean() * 100, 2)) + '%')])

,<lambda>
family_status,
Не женат / не замужем,9.75%
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.32%
женат / замужем,7.54%


In [22]:
dict(zip(df['family_status_id'], df['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

In [23]:
print('Создали словарь:')
family_dict = df[['family_status_id', 'family_status']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict)


print('\n\nСгруппированная таблица. Берем по id, другой столбец удалили:')
a = df.groupby('family_status_id')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])
display(a)


# Заменяем
print('\n\nЗаменяем численные значения по ключу словаря:')
a.reset_index().replace({'family_status_id': family_dict.family_status.to_dict()})

Создали словарь:


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем




Сгруппированная таблица. Берем по id, другой столбец удалили:


,count,sum,<lambda_0>
family_status_id,,,
0,12344,931,7.54%
1,4163,388,9.32%
2,959,63,6.57%
3,1195,85,7.11%
4,2810,274,9.75%




Заменяем численные значения по ключу словаря:


,family_status_id,count,sum,<lambda_0>
0,женат / замужем,12344,931,7.54%
1,гражданский брак,4163,388,9.32%
2,вдовец / вдова,959,63,6.57%
3,в разводе,1195,85,7.11%
4,Не женат / не замужем,2810,274,9.75%


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок есть.

Наблюдается довольно четкая корреляция:
* самые надежные плательщики - вдовы и вдовцы (6,57% должников);
* семейные и разведенные люди тоже показывают более высокую надежность по сравнению со средним значением (7.54% и 7.11% должников против 8.11 % в среднем по выборке);
* наименее надежны люди состоящие в гражданском браке (9.32% должников) или "неженатые" (9,75% должников). 

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Проанализируем значения в колонке `total_income`. Посчитаем уникальные значения (количество строк в выборке 21471):

In [324]:
len(df['total_income'].unique())

18608

Посмотрим основные метрики по колонке, чтоб выделять категории дохода более осмысленно:

In [325]:
print(str(df['total_income'].min()) + ' - минимум')
print(str(df['total_income'].max()) + ' - максимум')
print(str(round(df['total_income'].mean(), 1)) + ' - среднее')
print(str(df['total_income'].median()) + ' - медиана')

20667 - минимум
2265604 - максимум
165204.6 - среднее
141911.0 - медиана


Напишем функцию, которая будет принимать на вход доход и возвращать группу дохода:

In [326]:
def income_group(income):
    if income < 50000:
        return 'до 50_000'
    elif income < 100000:
        return 'от 50_000 до 100_000'
    elif income < 200000: # сюда войдут среднее и медианное значения
        return 'от 100_000 до 200_000'
    elif income < 500000:
        return 'от 200_000 до 500_000'
    elif income >= 500000:
        return 'свыше 500_000'
    else:
        return 'неизвестно'

Создадим новую колонку в датасете, в которую внесем данные о группе по доходу:

In [327]:
df['income_group'] = df['total_income'].apply(income_group)
display(df.head())

,id,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_group,purpose_id,children_group,income_group
0,0,1.0,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,один ребенок,от 200_000 до 500_000
1,1,1.0,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,4,один ребенок,от 100_000 до 200_000
2,2,0.0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,1,нет детей,от 100_000 до 200_000
3,3,3.0,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,"[дополнительный, , образование, \n]",получение образования,2,три ребенка,от 200_000 до 500_000
4,4,0.0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,3,нет детей,от 100_000 до 200_000


Сгруппируем заемщиков по группе дохода и вычислим процент должников в каждой группе.

In [328]:
df.groupby('income_group')['debt'].agg([lambda x: (str(round(x.mean() * 100, 2)) + '%')])

,<lambda>
income_group,
до 50_000,6.18%
от 100_000 до 200_000,8.62%
от 200_000 до 500_000,7.1%
от 50_000 до 100_000,8.09%
свыше 500_000,6.31%


Посмотрим на распределение заемщиков по группам по доходу:

In [329]:
df.groupby('income_group')['id'].agg([lambda x: (str(round(x.count() / len(df) * 100, 2)) + '%')])

,<lambda>
income_group,
до 50_000,1.73%
от 100_000 до 200_000,55.62%
от 200_000 до 500_000,22.56%
от 50_000 до 100_000,19.05%
свыше 500_000,1.03%


In [330]:
df.income_group.value_counts()

от 100_000 до 200_000    11942
от 200_000 до 500_000     4844
от 50_000 до 100_000      4091
до 50_000                  372
свыше 500_000              222
Name: income_group, dtype: int64

In [331]:
pd.qcut(df['total_income'], q=5)

0        (214546.0, 2265604.0]
1          (98554.0, 134155.0]
2         (134155.0, 162962.0]
3        (214546.0, 2265604.0]
4         (134155.0, 162962.0]
                 ...          
21520    (214546.0, 2265604.0]
21521     (134155.0, 162962.0]
21522     (20666.999, 98554.0]
21523    (214546.0, 2265604.0]
21524     (20666.999, 98554.0]
Name: total_income, Length: 21471, dtype: category
Categories (5, interval[float64]): [(20666.999, 98554.0] < (98554.0, 134155.0] < (134155.0, 162962.0] < (162962.0, 214546.0] < (214546.0, 2265604.0]]

In [332]:
pd.qcut(df['total_income'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])

0              высокий
1        ниже среднего
2              средний
3              высокий
4              средний
             ...      
21520          высокий
21521          средний
21522           низкий
21523          высокий
21524           низкий
Name: total_income, Length: 21471, dtype: category
Categories (5, object): [низкий < ниже среднего < средний < выше среднего < высокий]

**Вывод**

Зависимость сроков возврата кредита от уровня дохода есть, хотя сравнительно более слабая, чем по другим факторам.

Наименее надежные заемщики - люди с доходом от 100_000 до 200_000, которые составляют более половины всех заемщиков (55.62 %); среди них 8.62 % должников. 

Другая крупная группа заемщиков - люди с доходом от 50_000 до 100_000 имеют процент должников, соответствующий среднему по выборке (8.09 %, средний по выборке - 8.11 %).

Более надежными заемщиками являются люди с доходом свыше 200_000; процент должников среди них ниже (7.1 %), а сама группа выглядит перспективной - их 22.56 % от всех заемщиков. Доход выше 500_000 коррелирует с немного более низким процентом задолженности (6.31 %), однако эта группа заемщиков малочисленна.

Самый низкий процент должников (6.18 %) в группе заемщиков с доходом до 50_000, но она крайне малочисленна (1.73 % от всех заемщиков) и вряд ли это выгодные клиенты. Однако отказывать им в займе на основании низкого дохода не имеет смысла: надежность высокая, вероятные потери банка низкие.


### Как разные цели кредита влияют на его возврат в срок?

Сгруппируем заемщиков по категории цели кредита, подсчитаем их количество. Заемщиков с неопределенной категорией цели кредита нет.

In [333]:
df['purpose_group'].value_counts()

недвижимость             10207
автомобиль                4308
получение образования     4014
свадьба                   2335
ремонт жилья               607
Name: purpose_group, dtype: int64

Вычислим процент должников в каждой группе. 

In [334]:
df.groupby('purpose_group')['debt'].agg([lambda x: (str(round(x.mean() * 100, 2)) + '%')])

,<lambda>
purpose_group,
автомобиль,9.35%
недвижимость,7.32%
получение образования,9.22%
ремонт жилья,5.77%
свадьба,7.97%


**Вывод**

Зависимость между целью кредита и риском задолженности по нему есть. 

Наиболее высокий риск у кредитов на автомобиль и на образование (9.35 % и 9.22 % соответственно против средних 8.11 %). 

Наименьший процент должников среди заемщиков с кредитом на ремонт жилья (5.77 %).

Процент должников по кредитам на недвижимость и свадьбу можно оценить как средний (7.32 % и 7.97 % соответетвенно при среднем значении 8.09 %).

## Шаг 4. Общий вывод

На основании анализа обнаружены разной силы зависимости риска задолженности по кредиту от всех проанализированных факторов:
* наличия детей;
* семейного положения;
* уровня дохода (более слабая);
* цели кредита.

Самые надежные клиенты - это люди без детей (7,54% невозврата в срок). Заемщики с детьми в количестве больше десяти наименее надежны (10,53% невозврата в срок), что может быть связано не с детьми, а с национальными, культурными, социальными особенностями группы. Возможна также ошибка в данных, и на самом деле детей у этой группы заемщиков не 20, а 2. Заемщики с двумя детьми - вторая по ненадежности группа (процент невозврата в срок 9,53%); в случае, если в данных ошибка, процент невозврата в срок по этой группе еще выше.

Неженатые и живущие в гражданском браке заемщики менее надежны, чем женатые, разведенные и вдовые. Вдовцы меньше всего склонны к задолженностям по кредитам (6,57% невозврата в срок). Наиболее безответственные заемщики - неженатые (9,75% невозврата в срок).

Уровень дохода влияет слабее остальных факторов, при этом наименее надежна основная масса заемщиков - люди с доходом от 100_000 до 200_000 (8,62% невозврата в срок). Отказывать в получении кредита заемщикам с низким доходам не стоит - анализ показывает, что они менее остальных групп склонны к задолженностям (6,18% невозврата в срок). Вторая по надежности группа заемщиков - люди с доходом свыше 500_000 (6,31% невозврата в срок).

Наиболее рискованные цели кредита - кредиты на автомобиль (9,35%) и на образование (9,22%). Наиболее надежны кредиты на ремонт жилья (5,77% невозврата в срок).



Таким образом, не стоит давать кредит на автомобиль заемщику с 20 детьми, если он неженат или живет в гражданском браке. Также стоит быть осторожным, давая кредит на образование или машину заемщикам с двумя детьми, которые живут в гражданском браке или неженаты.

Надежнее и выгоднее всего дать кредит на ремонт жилья вдовцу или вдове с доходом от 500_000.

Заемщикам с низким доходом отказывать в выдаче кредита на этом основании не имеет смысла: они меньше других групп склонны к долгам. 